In [1]:
import pandas as pd
import os
import numpy as np

In [52]:
df = pd.read_csv("C:/Users/jacco/Documents/GitHub/Thesis/outputs/scores.csv")
df = df[df["algorithm"]!="MMHC"]
df

,items,nodes,timesteps,arcs_ratio,islands,values,algorithm,scorefunction,recall,precision,fscore,dist2opt,tp,tn,fp,fn
0,1000,4,1,0.999999,1,2,3OFF2,AIC,0.666667,1.000000,0.800000,0.333333,2,9,0,1
1,1000,4,1,0.999999,1,2,3OFF2,AIC,0.000000,0.000000,0.000000,1.414214,0,7,2,3
2,1000,4,1,0.999999,1,2,3OFF2,AIC,0.000000,0.000000,0.000000,1.414214,0,7,2,3
3,1000,4,1,0.999999,1,2,3OFF2,AIC,0.333333,0.500000,0.400000,0.833333,1,8,1,2
4,1000,4,1,0.999999,1,2,3OFF2,AIC,0.000000,0.000000,0.000000,1.414214,0,7,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,10000,8,3,2.000000,1,4,TABU,K2,0.527273,0.467742,0.495726,0.711878,29,464,33,26
129786,10000,8,3,2.000000,1,4,TABU,K2,0.672727,0.521127,0.587302,0.580023,37,463,34,18
129787,10000,8,3,2.000000,1,4,TABU,K2,0.890909,0.777778,0.830508,0.247555,49,483,14,6
129788,10000,8,3,2.000000,1,4,TABU,K2,1.000000,0.785714,0.880000,0.214286,55,482,15,0


In [57]:
nodes = [4,6,8,10]
timesteps = [1,2,3]
datapoints = [10000]
values = [2, 4]
# TODO: MMHC
methods = ['HC', 'TABU', 'MMHC', '3OFF2', 'MIIC']
scorefunctions = ['BDEU', 'AIC', 'BIC', 'K2', 'L2L']

arcs_ratio = [0.999999, 1.25, 1.5, 1.75, 2.]

parameters = [(r, n, t, d, v, m, f) for r in arcs_ratio for n in nodes for t in timesteps for d in datapoints for v in values for m in methods for f in scorefunctions]
parameters_df = pd.DataFrame(parameters, columns=["arcs_ratio", "nodes","timesteps","items","values","algorithm","scorefunction"])

grouped = df.groupby(["items","nodes","timesteps","arcs_ratio","values","algorithm","scorefunction"]).count()

done = pd.merge(parameters_df, grouped, on=["items","nodes","timesteps","arcs_ratio","values","algorithm","scorefunction"], how="outer")
done.to_csv("done.csv")
done

,arcs_ratio,nodes,timesteps,items,values,algorithm,scorefunction,islands,recall,precision,fscore,dist2opt,tp,tn,fp,fn
0,0.999999,4,1,10000,2,HC,BDEU,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
1,0.999999,4,1,10000,2,HC,AIC,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
2,0.999999,4,1,10000,2,HC,BIC,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
3,0.999999,4,1,10000,2,HC,K2,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
4,0.999999,4,1,10000,2,HC,L2L,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7117,2.000000,8,3,9000,4,MIIC,L2L,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
7118,2.000000,8,3,9000,4,TABU,AIC,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7119,2.000000,8,3,9000,4,TABU,BDEU,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
7120,2.000000,8,3,9000,4,TABU,BIC,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0


In [64]:
r, n, t, v, m, f = 1.5,4,2,2,"MMHC","BIC"
done[((done["arcs_ratio"]==r)&(done["nodes"]==n)&(done["timesteps"]==t)&(done["values"]==v)&(done["algorithm"]==m)&(done["scorefunction"]==f))]

,arcs_ratio,nodes,timesteps,items,values,algorithm,scorefunction,islands,recall,precision,fscore,dist2opt,tp,tn,fp,fn
1262,1.5,4,2,10000,2,MMHC,BIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
batches = 30
parameters = pd.read_csv("done.csv")
for i, row in parameters.iterrows():
    if row["recall"] < batches and not np.isnan(row["recall"]):
        for batch in range(int(row["recall"]),batches):
            print("notna",batch)
    elif np.isnan(row["recall"]):
        for batch in range(batches):
            print("welna",batch)

notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
not

welna 25
welna 26
welna 27
welna 28
welna 29
welna 0
welna 1
welna 2
welna 3
welna 4
welna 5
welna 6
welna 7
welna 8
welna 9
welna 10
welna 11
welna 12
welna 13
welna 14
welna 15
welna 16
welna 17
welna 18
welna 19
welna 20
welna 21
welna 22
welna 23
welna 24
welna 25
welna 26
welna 27
welna 28
welna 29
welna 0
welna 1
welna 2
welna 3
welna 4
welna 5
welna 6
welna 7
welna 8
welna 9
welna 10
welna 11
welna 12
welna 13
welna 14
welna 15
welna 16
welna 17
welna 18
welna 19
welna 20
welna 21
welna 22
welna 23
welna 24
welna 25
welna 26
welna 27
welna 28
welna 29
welna 0
welna 1
welna 2
welna 3
welna 4
welna 5
welna 6
welna 7
welna 8
welna 9
welna 10
welna 11
welna 12
welna 13
welna 14
welna 15
welna 16
welna 17
welna 18
welna 19
welna 20
welna 21
welna 22
welna 23
welna 24
welna 25
welna 26
welna 27
welna 28
welna 29
welna 0
welna 1
welna 2
welna 3
welna 4
welna 5
welna 6
welna 7
welna 8
welna 9
welna 10
welna 11
welna 12
welna 13
welna 14
welna 15
welna 16
welna 17
welna 18
welna 19
welna

notna 28
notna 29
notna 1
notna 2
notna 3
notna 4
notna 5
notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 1
notna 2
notna 3
notna 4
notna 5
notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 28
notna 29
notna 1
notna 2
notna 3
notna 4
notna 5
notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notn

notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
notna 23
notna 24
notna 25
notna 26
notna 27
notna 28
notna 29
notna 6
notna 7
notna 8
notna 9
notna 10
notna 11
notna 12
notna 13
notna 14
notna 15
notna 16
notna 17
notna 18
notna 19
notna 20
notna 21
notna 22
not

NameError: name 'run_sims' is not defined

In [26]:
files = os.listdir("C:/Users/jacco/Documents/GitHub/Thesis/outputs/structures/")
for file in files:
    if file[:-7]+".csv" not in os.listdir("C:/Users/jacco/Documents/GitHub/Thesis/outputs/scores/"):
        os.remove("C:/Users/jacco/Documents/GitHub/Thesis/outputs/structures/"+file)
        print(file[:-7]+".csv")

structure1234.csv
structure2254_18.csv
structure2259_6.csv
structure2260_0.csv
structure2260_1.csv
structure2260_10.csv
structure2260_11.csv
structure2260_12.csv
structure2260_13.csv
structure2260_14.csv
structure2260_15.csv
structure2260_16.csv
structure2260_17.csv
structure2260_18.csv
structure2260_19.csv
structure2260_2.csv
structure2260_20.csv
structure2260_21.csv
structure2260_22.csv
structure2260_23.csv
structure2260_24.csv
structure2260_25.csv
structure2260_26.csv
structure2260_27.csv
structure2260_28.csv
structure2260_29.csv
structure2260_3.csv
structure2260_4.csv
structure2260_5.csv
structure2260_6.csv
structure2260_7.csv
structure2260_8.csv
structure2260_9.csv
structure2261_0.csv
structure2261_1.csv
structure2261_10.csv
structure2261_11.csv
structure2261_12.csv
structure2261_13.csv
structure2261_14.csv
structure2261_15.csv
structure2261_16.csv
structure2261_17.csv
structure2261_18.csv
structure2261_19.csv
structure2261_2.csv
structure2261_20.csv
structure2261_21.csv
structure

structure2335_2.csv
structure2335_20.csv
structure2335_21.csv
structure2335_22.csv
structure2335_23.csv
structure2335_24.csv
structure2335_25.csv
structure2335_26.csv
structure2335_27.csv
structure2335_28.csv
structure2335_29.csv
structure2335_3.csv
structure2335_4.csv
structure2335_5.csv
structure2335_6.csv
structure2335_7.csv
structure2335_8.csv
structure2335_9.csv
structure2336_0.csv
structure2336_1.csv
structure2336_2.csv
structure2336_3.csv
structure2336_4.csv
structure2336_5.csv
structure2336_6.csv
structure2336_7.csv
structure2337_0.csv
structure2337_1.csv
structure2337_10.csv
structure2337_11.csv
structure2337_12.csv
structure2337_13.csv
structure2337_14.csv
structure2337_15.csv
structure2337_16.csv
structure2337_17.csv
structure2337_18.csv
structure2337_19.csv
structure2337_2.csv
structure2337_20.csv
structure2337_21.csv
structure2337_22.csv
structure2337_23.csv
structure2337_24.csv
structure2337_25.csv
structure2337_26.csv
structure2337_27.csv
structure2337_28.csv
structure23

structure2411_27.csv
structure2411_28.csv
structure2411_3.csv
structure2411_4.csv
structure2411_5.csv
structure2411_6.csv
structure2411_7.csv
structure2411_8.csv
structure2411_9.csv
structure2412_0.csv
structure2412_1.csv
structure2412_10.csv
structure2412_11.csv
structure2412_12.csv
structure2412_13.csv
structure2412_14.csv
structure2412_15.csv
structure2412_16.csv
structure2412_17.csv
structure2412_18.csv
structure2412_19.csv
structure2412_2.csv
structure2412_20.csv
structure2412_21.csv
structure2412_22.csv
structure2412_23.csv
structure2412_24.csv
structure2412_25.csv
structure2412_26.csv
structure2412_27.csv
structure2412_28.csv
structure2412_29.csv
structure2412_3.csv
structure2412_4.csv
structure2412_5.csv
structure2412_6.csv
structure2412_7.csv
structure2412_8.csv
structure2412_9.csv
structure2413_0.csv
structure2413_1.csv
structure2413_10.csv
structure2413_11.csv
structure2413_12.csv
structure2413_13.csv
structure2413_14.csv
structure2413_15.csv
structure2413_16.csv
structure24

structure2489_0.csv
structure2501_2.csv
structure2504_0.csv
structure2506_2.csv
structure2509_0.csv
structure2510_0.csv
structure2510_1.csv
structure2510_10.csv
structure2510_11.csv
structure2510_12.csv
structure2510_13.csv
structure2510_14.csv
structure2510_15.csv
structure2510_16.csv
structure2510_17.csv
structure2510_18.csv
structure2510_19.csv
structure2510_2.csv
structure2510_20.csv
structure2510_21.csv
structure2510_22.csv
structure2510_23.csv
structure2510_24.csv
structure2510_25.csv
structure2510_26.csv
structure2510_27.csv
structure2510_28.csv
structure2510_29.csv
structure2510_3.csv
structure2510_4.csv
structure2510_5.csv
structure2510_6.csv
structure2510_7.csv
structure2510_8.csv
structure2510_9.csv
structure2511_0.csv
structure2511_1.csv
structure2511_2.csv
structure2512_0.csv
structure2512_1.csv
structure2512_10.csv
structure2512_11.csv
structure2512_12.csv
structure2512_13.csv
structure2512_14.csv
structure2512_15.csv
structure2512_16.csv
structure2512_17.csv
structure251

structure2588_4.csv
structure2588_5.csv
structure2588_6.csv
structure2588_7.csv
structure2588_8.csv
structure2588_9.csv
structure2589_0.csv
structure2591_11.csv
structure2592_11.csv
structure2594_11.csv
structure2596_11.csv
structure2597_11.csv
structure2599_11.csv
structure2601_0.csv
structure2604_0.csv
structure2606_0.csv
structure2607_12.csv
structure2609_0.csv
structure2610_0.csv
structure2610_1.csv
structure2610_10.csv
structure2610_11.csv
structure2610_12.csv
structure2610_13.csv
structure2610_14.csv
structure2610_15.csv
structure2610_16.csv
structure2610_17.csv
structure2610_18.csv
structure2610_19.csv
structure2610_2.csv
structure2610_20.csv
structure2610_21.csv
structure2610_22.csv
structure2610_23.csv
structure2610_24.csv
structure2610_25.csv
structure2610_26.csv
structure2610_27.csv
structure2610_28.csv
structure2610_29.csv
structure2610_3.csv
structure2610_4.csv
structure2610_5.csv
structure2610_6.csv
structure2610_7.csv
structure2610_8.csv
structure2610_9.csv
structure2611

structure67561_12.csv
structure67561_13.csv
structure67561_14.csv
structure67561_15.csv
structure67561_16.csv
structure67561_17.csv
structure67561_18.csv
structure67561_19.csv
structure67561_2.csv
structure67561_20.csv
structure67561_21.csv
structure67561_22.csv
structure67561_23.csv
structure67561_24.csv
structure67561_25.csv
structure67561_26.csv
structure67561_27.csv
structure67561_28.csv
structure67561_29.csv
structure67561_3.csv
structure67561_4.csv
structure67561_5.csv
structure67561_6.csv
structure67561_7.csv
structure67561_8.csv
structure67561_9.csv
structure67562_0.csv
structure67562_1.csv
structure67562_10.csv
structure67562_11.csv
structure67562_12.csv
structure67562_13.csv
structure67562_14.csv
structure67562_15.csv
structure67562_16.csv
structure67562_17.csv
structure67562_18.csv
structure67562_19.csv
structure67562_2.csv
structure67562_20.csv
structure67562_21.csv
structure67562_22.csv
structure67562_23.csv
structure67562_24.csv
structure67562_25.csv
structure67562_3.csv


structure67661_19.csv
structure67661_2.csv
structure67661_20.csv
structure67661_21.csv
structure67661_22.csv
structure67661_23.csv
structure67661_24.csv
structure67661_25.csv
structure67661_26.csv
structure67661_27.csv
structure67661_28.csv
structure67661_29.csv
structure67661_3.csv
structure67661_4.csv
structure67661_5.csv
structure67661_6.csv
structure67661_7.csv
structure67661_8.csv
structure67661_9.csv
structure67662_0.csv
structure67662_1.csv
structure67662_10.csv
structure67662_11.csv
structure67662_12.csv
structure67662_13.csv
structure67662_14.csv
structure67662_15.csv
structure67662_16.csv
structure67662_17.csv
structure67662_18.csv
structure67662_19.csv
structure67662_2.csv
structure67662_20.csv
structure67662_21.csv
structure67662_22.csv
structure67662_23.csv
structure67662_24.csv
structure67662_25.csv
structure67662_26.csv
structure67662_27.csv
structure67662_28.csv
structure67662_29.csv
structure67662_3.csv
structure67662_4.csv
structure67662_5.csv
structure67662_6.csv
str